In [2]:
import sys

print(sys.version)

3.8.10 (default, Nov 14 2021, 21:32:59) 
[Clang 12.0.5 (clang-1205.0.22.9)]


### Spark 환경 설정

로컬 머신 내 다운받은 SPARK_HOME 경로를 정확히 지정합니다. findspark() 를 통해 경로 내 Spark 를 실행할 수 있습니다.
만약 AWS 환경이라면 EFS 등의 공유 스토리지에 Hadoop, Java, Spark 환경을 버전별로 구비해놓고 Read-only 로 마운트 해 사용할 수 있습니다.

In [9]:
DATASET_ROOT = "/Users/kun/github/1ambda/practical-data-pipeline-code"
SPARK_HOME = "/Users/kun/github/spark/spark-3.2.0-bin-hadoop3.2"

In [10]:
import findspark

findspark.init(SPARK_HOME)
#findspark.add_packages(["org.apache.hadoop:hadoop-aws:3.2.2", "com.amazonaws:aws-java-sdk-bundle:1.11.375"])

### Spark Session 생성

로컬모드에서 실행할 Spark Session 을 만듭니다. (`.master("local[*]")`)
- 일반적인 Spark 설정은 `$SPARK_HOME/conf/spark-defaults.conf` 내에서 세팅해 공통환경으로 사용합니다. 다만 이 예제에서는 보여주기 위해 SparkConf 를 이용해 설정합니다.
- Hive Metastore URI 등 HMS 관련 설정은 `$SPARK_HOME/conf/hive-site.conf` 내에서 세팅해 공통 환경으로 사용합니다.
- 이 예제에서는 Minio 를 사용하므로 Access Key, Secret Key 를 사용합니다. AWS 위에서 실행된다면 [AWS Instance Profile](https://docs.aws.amazon.com/ko_kr/IAM/latest/UserGuide/id_roles_use_switch-role-ec2_instance-profiles.html) 을 이용할 수 있으므로 키를 세팅하지 않습니다.

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("example-app") \
    .enableHiveSupport() \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/01 14:51:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/01 14:51:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Spark UI 확인

http://localhost:4040 에서 Spark UI 를 확인할 수 있습니다.  
만약 아래와 같은 메세지가 Spark Session 을 초기화 하는 과정에서 보였다면 Port 를 4041 (http://localhost:4041) 로 변경해 브라우저에서 확인합니다.

```
21/12/01 14:51:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
```

  
  

Production 환경에서는 사용자의 Jupyter Container (Kubernetes) 내에 https://github.com/jupyterhub/jupyter-server-proxy 등을 통해 제공할 수 있습니다.

### Spark 설정 확인

In [6]:
spark.sparkContext.getConf().getAll()

[('spark.app.id', 'local-1638337901493'),
 ('spark.driver.port', '64469'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '192.168.0.2'),
 ('spark.app.startTime', '1638337900092'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir',
  'file:/Users/kun/github/1ambda/practical-data-pipeline-code/_notebook/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'example-app')]

### 데이터 읽기

Ecommerce 데이터셋 (약 2.4 GiB, 2천만 Rows) 를 읽습니다.
만약 로컬 머신의 메모리가 부족하면 `_datasets/ecommerce` 파일 내 1 개의 CSV 만 남기고 나머지는 삭제해도 괜찮습니다.

In [15]:
df = spark.read.load(f"{DATASET_ROOT}/_datasets/ecommerce/*.csv", 
                     format="csv", inferSchema="true", header="true")

In [16]:
df.count()

20692840